# Assess income level predictions on adult census data

This notebook demonstrates the use of the `responsibleai` API to assess a model trained on census data. It walks through the API calls necessary to create a widget with model analysis insights, then guides a visual analysis of the model.

* [Launch Responsible AI Toolbox](#Launch-Responsible-AI-Toolbox)
    * [Train a Model](#Train-a-Model)
    * [Create Model and Data Insights](#Create-Model-and-Data-Insights)
* [Assess Your Model](#Assess-Your-Model)
    * [Aggregate Analysis](#Aggregate-Analysis)
    * [Individual Analysis](#Individual-Analysis)

## Launch Responsible AI Toolbox

The following section examines the code necessary to create datasets and a model. It then generates insights using the `responsibleai` API that can be visually analyzed.

### Train a Model
*The following section can be skipped. It loads a dataset and trains a model for illustrative purposes.*

In [ ]:
!pip install raiutils
!pip install raiwidgets
!pip install interpret

import zipfile

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

import pandas as pd
from lightgbm import LGBMClassifier

#!pip install prompt-toolkit==1.0.15
#!pip install ipykernel==4.10
#!pip install imgaug==0.2.6


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached ipykernel-6.15.1-py3-none-any.whl (132 kB)
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 4.10.0
    Uninstalling ipykernel-4.10.0:
      Successfully uninstalled ipykernel-4.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
responsibleai 0.19.0 requires ipykernel<=6.6.0, but you have ipykernel 6.15.1 which is incompatible.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.30 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.15.1 which is

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


First, load the census dataset and specify the different types of features. Compose a pipeline which contains a preprocessor and estimator.

In [ ]:
from raiutils.dataset import fetch_dataset
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

def split_label(dataset, target_feature):
    X = dataset.drop([target_feature], axis=1)
    y = dataset[[target_feature]]
    return X, y

csv_columns = [
  "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
  "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
  "hours-per-week", "country", "income"]


target_feature = 'income'
categorical_features = ['workclass', 'education', 'marital-status',
                        'occupation', 'relationship', 'race', 'sex', 'country']


train_data_path = "https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/adult.data"
test_data_path = "https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/adult.test"
train_data = pd.read_csv(train_data_path, names=csv_columns, skipinitialspace=True)
test_data = pd.read_csv(test_data_path, names=csv_columns, skipinitialspace=True, skiprows=1)
test_data["income"] = test_data["income"].apply(lambda x: x[:-1]) #remove weird period at the end of income col in this this data file

In [ ]:
X_train_original, y_train = split_label(train_data, target_feature)
X_test_original, y_test = split_label(test_data, target_feature)

y_train = y_train[target_feature].to_numpy()
y_test = y_test[target_feature].to_numpy()


# Take 500 samples from the test data
test_data_sample = test_data.sample(n=500, random_state=5)

Train the classification pipeline composed in the previous cell on the training data.

In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier
ebm = ExplainableBoostingClassifier()
model = ebm.fit(X_train_original, y_train)

### Create Model and Data Insights

In [ ]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


To use Responsible AI Toolbox, initialize a RAIInsights object upon which different components can be loaded.

RAIInsights accepts the model, the full dataset, the test dataset, the target feature string, the task type string, and a list of strings of categorical feature names as its arguments.

In [ ]:
rai_insights = RAIInsights(model, train_data, test_data_sample, target_feature, 'classification',
                           categorical_features=categorical_features)

Add the components of the toolbox that are focused on model assessment.

In [ ]:
# Interpretability
rai_insights.explainer.add()
# Error Analysis
rai_insights.error_analysis.add()
# Counterfactuals: accepts total number of counterfactuals to generate, the label that they should have, and a list of 
                # strings of categorical feature names
rai_insights.counterfactual.add(total_CFs=10, desired_class='opposite')

Once all the desired components have been loaded, compute insights on the test set.

In [ ]:
rai_insights.compute()

100%|██████████| 500/500 [02:10<00:00,  3.84it/s]
categorical_feature in Dataset is overridden.
New categorical_feature is [1, 3, 5, 6, 7, 8, 9, 13]
categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.


Compose some cohorts which can be injected into the `ResponsibleAIDashboard`.

In [ ]:
from raiwidgets.cohort import Cohort, CohortFilter, CohortFilterMethods

# Cohort on age and hours-per-week features in the dataset
cohort_filter_age = CohortFilter(
    method=CohortFilterMethods.METHOD_LESS,
    arg=[65],
    column='age')
cohort_filter_hours_per_week = CohortFilter(
    method=CohortFilterMethods.METHOD_GREATER,
    arg=[40],
    column='hours-per-week')

user_cohort_age_and_hours_per_week = Cohort(name='Cohort Age and Hours-Per-Week')
user_cohort_age_and_hours_per_week.add_cohort_filter(cohort_filter_age)
user_cohort_age_and_hours_per_week.add_cohort_filter(cohort_filter_hours_per_week)

# Cohort on marital-status feature in the dataset
cohort_filter_marital_status = CohortFilter(
    method=CohortFilterMethods.METHOD_INCLUDES,
    arg=["Never-married", "Divorced"],
    column='marital-status')

user_cohort_marital_status = Cohort(name='Cohort Marital-Status')
user_cohort_marital_status.add_cohort_filter(cohort_filter_marital_status)

# Cohort on index of the row in the dataset
cohort_filter_index = CohortFilter(
    method=CohortFilterMethods.METHOD_LESS,
    arg=[20],
    column='Index')

user_cohort_index = Cohort(name='Cohort Index')
user_cohort_index.add_cohort_filter(cohort_filter_index)

# Cohort on predicted target value
cohort_filter_predicted_y = CohortFilter(
    method=CohortFilterMethods.METHOD_INCLUDES,
    arg=['>50K'],
    column='Predicted Y')

user_cohort_predicted_y = Cohort(name='Cohort Predicted Y')
user_cohort_predicted_y.add_cohort_filter(cohort_filter_predicted_y)

# Cohort on predicted target value
cohort_filter_true_y = CohortFilter(
    method=CohortFilterMethods.METHOD_INCLUDES,
    arg=['>50K'],
    column='True Y')

user_cohort_true_y = Cohort(name='Cohort True Y')
user_cohort_true_y.add_cohort_filter(cohort_filter_true_y)

cohort_list = [user_cohort_age_and_hours_per_week,
               user_cohort_marital_status,
               user_cohort_index,
               user_cohort_predicted_y,
               user_cohort_true_y]

Finally, visualize and explore the model insights. Use the resulting widget or follow the link to view this in a new tab.

In [ ]:
rai = ResponsibleAIDashboard(rai_insights, cohort_list=cohort_list)

Error checking for gevent monkey-patching.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_utils.py", line 278, in notify_about_gevent_if_needed
    saved = gevent_monkey.saved
AttributeError: module 'gevent.monkey' has no attribute 'saved'

PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/gevent/threadpool.py", line 157, in _before_run_task
    _sys.settrace(_get_thread_trace())


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/0

ResponsibleAI started at http://localhost:5000


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)")) #Make sure the port number here is the same as whatever the localhost port is

https://wjxgrzpp3qn-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from raiwidgets import ErrorAnalysisDashboard
ea_db = ErrorAnalysisDashboard(dataset=X_test_original, true_y=y_test)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5001)"))

could not convert string to float: 'Private'


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/raiwidgets/error_analysis_dashboard_input.py", line 498, in importances
    scores = self._error_analyzer.compute_importances()
  File "/usr/local/lib/python3.7/dist-packages/erroranalysis/analyzer/error_analyzer.py", line 386, in compute_importances
    return mutual_info_classif(input_data, diff).tolist()
  File "/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_mutual_info.py", line 464, in mutual_info_classif
    return _estimate_mi(X, y, discrete_features, True, n_neighbors, copy, random_state)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_mutual_info.py", line 256, in _estimate_mi
    X, y = check_X_y(X, y, accept_sparse="csc", y_numeric=not discrete_target)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py", line 976, in check_X_y
    estimator=estimator,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py",

https://pmg7swjzb48-496ff2e9c6d22116-5001-colab.googleusercontent.com/


In [ ]:
!python -m http.server 8000
# Once you've run this cell, click on the link output from the cell above this one. It should show the widget. 
# If there's an error and the error message says "port already in use", try a different port number here, as long as it's not whatever localhost port above is. 

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...


In [ ]:
# The code in the two cells below helps host whatever is on localhost in a cell here. If we can figure out a good height parameter value, that would be another way to go

# from IPython.display import Javascript

# def show_port(port, height=400):
#   display(Javascript("""
#   (async ()=>{
#     fm = document.createElement('iframe')
#     fm.src = await google.colab.kernel.proxyPort(%s)
#     fm.width = '95%%'
#     fm.height = '%d'
#     fm.frameBorder = 0
#     document.body.append(fm)
#   })();
#   """ % (port, height) ))

In [ ]:
# get_ipython().system_raw('python3 -m http.server 8000 &') 
# show_port(5000)

## Assess Your Model

### Aggregate Analysis

The Error Analysis component is displayed at the top of the dashboard widget. To visualize how error is broken down across cohorts, use the tree map view to understand how it filters through the nodes.

![Error Analysis tree map with "Marital Status == 2," "Capital Gain <= 1287.5," "Capital Loss <= 1494.5" path selected](https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/EA_img/classification-assessment-1.png)

Over 40% of the error in this model is concentrated in datapoints of people who are married, have higher education and minimal capital gain. 

Let's see what else we can discover about this cohort.

First, save the cohort by clicking "Save as a new cohort" on the right side panel of the Error Analysis component.

![Cohort creation sidebar and tree map cohort creation popup](https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/EA_img/classification-assessment-2.png)

To switch to this cohort for analysis, click "Switch global cohort" and select the recently saved cohort from the dropdown.

![Popup with dropdown to shift cohort from "All data" to "Married, Low Capital Loss/Gain" accompanied by cohort statistics](https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/EA_img/classification-assessment-3.png)

The Model Overview component allows the comparison of statistics across multiple saved cohorts.

The diagram indicates that the model is misclassifying datapoints of married individuals with low capital gains and high education as lower income (false negative).

![Bar chart of classification outcomes (true negative, true positive, false negative, false positive) compared across cohorts](https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/EA_img/classification-assessment-4.png)

Looking at the ground truth statistics of the overall data and the erroneous cohort, we realize there are opposite patterns in terms of high income representation in ground truth. While the overall data is representing more individuals with actual income of <= 50K, the married individuals with low capital gains and high education represent more individuals with actual income of > 50K. Given the small size of the dataset and this reverse pattern, the model makes more mistakes in predicting high income individuals. One action item is to collect a lot more data in both cohorts and retrain the model.

![image-3.png](https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/EA_img/classification-assessment-5.png)

![image.png](https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/EA_img/classification-assessment-6.png)

The Interpretability component displays feature importances for model predictions at an individual and aggregate level. The plot below indicates that the `marital-status` attribute influence model predictions the most on average.

![Top 5 features of the cohort, in descending importance: relationship, age, capital gain, education-num, hours per week](https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/EA_img/classification-assessment-7.png)

The lower half of this tab specifies how marita status affects model prediction. Being a husband or wife (married-civ-spouse) is more likely to pull the prediction away from <=50k, possibly because couples have a higher cumulative income.

![Feature importance stratified by relationship](https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/EA_img/classification-assessment-8.png)

### Individual Analysis

Let's revisit Data Explorer. In the "Individual datapoints" view, we can see the prediction probabilities of each point. Point 510 is one that was just above the threshold to be classified as income of > 50K.

![Scatter plot of prediction probabilities (rounded to 0.2) on the y-axis and index on the x-axis](https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/EA_img/classification-assessment-9.png)

What factors led the model to make this decision?

The "Individual feature importance" tab in the Interpretability component's Feature Importances section let you select points for further analysis.

![Table of datapoints with row 510 selected](https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/EA_img/classification-assessment-10.png)

Under this, the feature importance plot shows `capital-gain` and `native-country` as the most significant factors leading to the <= 50K classification. Changing these may cause the threshold to be crossed and the model to predict the opposite class. Please note that depending on the context, the high importance of `native-country` might be considered as a fairness issue.

![Feature importance plot for classification of 0 (descending, positive to negative): age, hours per week, capital gain, race, education-num, workclass, sex, country, occupation, marital status, relationship, capital loss](https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/EA_img/classification-assessment-11.png)

The What-If Counterfactuals component focuses on how to change features slightly in order to change model predictions. As seen in its top ranked features bar plot, changing this person's marital-status, capital-loss, and education-num have the highest impact on flipping the prediction to > 50K.

![Top-ranked features (descending) for datapoint 510 to perturb to flip model prediction: age, hours per week, capital gain, capital loss, marital status, occupation, education-num, workclass, relationship, race, sex, country](https://raw.githubusercontent.com/davisjrule/satisficing-main-study/main/EA_img/classification-assessment-12.png)